In [1]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True)

![png](Data/movies.png)

In [2]:
# Confirm folder was created and files added successfully
os.listdir("Data/")

['.ipynb_checkpoints',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'movies.png',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json']

In [3]:
!pip install pandasql
import pandas as pd
import numpy as np

In [4]:
basics = pd.read_csv('Data/title.basics.tsv.gz', sep='\t', low_memory=False)
basics.head(1)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"


In [5]:
ratings = pd.read_csv('Data/title.ratings.tsv.gz', sep='\t', low_memory=False)

In [6]:
akas = pd.read_csv('Data/title-akas-us-only.csv', low_memory=False)

In [7]:
akas.head(5)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [8]:
basics.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
ratings.head(5)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1988
1,tt0000002,5.8,265
2,tt0000003,6.5,1849
3,tt0000004,5.5,178
4,tt0000005,6.2,2632


In [10]:
# Filtering non-us movies
filter_us_titles = basics['tconst'].isin(akas['titleId'])
basics = basics[filter_us_titles]

In [11]:
ratings = ratings[filter_us_titles]

C:\Users\James\AppData\Local\Temp\ipykernel_12808\1531508872.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ratings = ratings[filter_us_titles]


In [12]:
basics = basics.replace({'\\N':np.nan})
ratings = ratings.replace({'\\N':np.nan})
akas = akas.replace({'\\N':np.nan})

In [13]:
basics.isnull().sum()

tconst                  0
titleType               0
primaryTitle            2
originalTitle           2
isAdult                 0
startYear           98665
endYear           1328513
runtimeMinutes     503119
genres              28616
dtype: int64

In [14]:
basics['runtimeMinutes'].value_counts()

runtimeMinutes
60      39252
30      36784
5       25015
10      24007
7       22542
        ...  
834         1
2580        1
1620        1
1271        1
462         1
Name: count, Length: 703, dtype: int64

In [15]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)
basics.dropna(subset=['genres'], inplace=True)

In [16]:
filter_documentaries = basics['genres'].str.contains('Documentary')

In [17]:
basics = basics[~filter_documentaries]

In [18]:
basics['startYear'].value_counts()

startYear
2018    35761
2017    34161
2016    33687
2015    32770
2014    32481
        ...  
1890        2
2029        2
1887        2
1893        1
1881        1
Name: count, Length: 140, dtype: int64

In [19]:
basics['startYear'].unique()

array(['1892', '1893', '1894', '1895', '1896', '1897', '1898', '1899',
       '1900', '1901', '1902', '1903', '1904', '1905', '1912', '1907',
       '1906', '1908', '1909', '1910', '1911', '1913', '1919', '1914',
       '1916', '1915', '1936', '1917', '1925', '1918', '1921', '1920',
       '1922', '1923', '1924', '1927', '1926', '1935', '1929', '1928',
       '1942', '1930', '1934', '1931', '1932', '1937', '1950', nan,
       '1933', '1938', '1939', '1945', '1940', '1946', '1944', '1949',
       '1957', '1943', '1941', '1948', '2001', '1953', '1954', '1983',
       '1947', '2021', '1952', '1995', '1951', '1955', '1962', '1956',
       '1961', '1970', '1958', '1959', '2012', '1960', '1965', '1963',
       '1988', '1964', '1971', '1969', '1966', '1967', '1968', '1990',
       '1973', '1979', '1976', '1974', '2020', '1978', '1972', '1977',
       '1989', '1975', '1985', '1981', '1986', '1987', '2018', '1992',
       '1980', '1984', '1982', '1991', '1993', '1999', '2005', '1994',
       '1

In [20]:
import pandas as pd

# Convert 'nan' strings to NaN (missing values)
basics['startYear'] = pd.to_numeric(basics['startYear'], errors='coerce')

# Drop rows with NaN values in the 'startYear' column
basics = basics.dropna(subset=['startYear'])

# Now, you can filter the DataFrame as before
filtered_movies = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022) & (basics['titleType'] == 'movie')]

In [21]:
basics['startYear'] = pd.to_numeric(basics['startYear'], errors='coerce')

In [22]:
basics.dropna(subset=['startYear'])

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894.0,NaN,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894.0,NaN,1,"Short,Sport"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894.0,NaN,45,Romance
...,...,...,...,...,...,...,...,...,...
10016704,tt9916214,short,Drown the Clown,Drown the Clown,0,2019.0,NaN,8,"Drama,Short"
10016724,tt9916254,video,Big Tit Cream Pie 32,Big Tit Cream Pie 32,1,2015.0,NaN,226,Adult
10016770,tt9916348,video,Ancient World Exposed,Ancient World Exposed,0,2019.0,NaN,67,History
10016777,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History"


In [23]:
# Convert 'startYear' column to integer
basics['startYear'] = basics['startYear'].astype(int)

# Filter the DataFrame
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022) & (basics['titleType'] == 'movie')]

In [24]:
basics['startYear'].unique()

array([2001, 2020, 2018, 2005, 2002, 2017, 2006, 2004, 2000, 2009, 2008,
       2007, 2022, 2003, 2011, 2021, 2010, 2012, 2013, 2016, 2014, 2015,
       2019])

In [25]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86979 entries, 34802 to 10016777
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          86979 non-null  object
 1   titleType       86979 non-null  object
 2   primaryTitle    86978 non-null  object
 3   originalTitle   86978 non-null  object
 4   isAdult         86979 non-null  object
 5   startYear       86979 non-null  int32 
 6   endYear         0 non-null      object
 7   runtimeMinutes  86979 non-null  object
 8   genres          86979 non-null  object
dtypes: int32(1), object(8)
memory usage: 6.3+ MB


In [26]:
# Drop the 'endYear' column
basics = basics.drop(columns=['endYear'])

In [27]:
basics.dropna()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,70,Drama
67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126,Drama
...,...,...,...,...,...,...,...,...
10016149,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,74,Drama
10016544,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,97,"Comedy,Drama,Fantasy"
10016684,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
10016693,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"


In [28]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86979 entries, 34802 to 10016777
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          86979 non-null  object
 1   titleType       86979 non-null  object
 2   primaryTitle    86978 non-null  object
 3   originalTitle   86978 non-null  object
 4   isAdult         86979 non-null  object
 5   startYear       86979 non-null  int32 
 6   runtimeMinutes  86979 non-null  object
 7   genres          86979 non-null  object
dtypes: int32(1), object(7)
memory usage: 5.6+ MB


In [29]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 370233 entries, 0 to 1331484
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         370233 non-null  object 
 1   averageRating  370233 non-null  float64
 2   numVotes       370233 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.3+ MB


In [30]:
ratings.to_csv('ratings.csv', index=False)

In [31]:
basics.to_csv('bascis.csv', index=False)